In [ ]:
import os
import pandas as pd
import numpy as np
from glob import glob
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

EDA

In [ ]:
path = '../input/petfinder-pawpularity-score/'
train = pd.read_csv(path + 'train.csv')
test= pd.read_csv(path + 'test.csv')
train_jpg = glob(path + "train/*.jpg")
test_jpg = glob(path + "test/*.jpg")

In [ ]:
train.head()

In [ ]:
for i in range(5):
    image = plt.imread(train_jpg[i]) 
    print(image.shape)
    image_show= plt.imread(train_jpg[i]) 
    plt.imshow(image_show)
    plt.show()

In [ ]:
id_modified=train_jpg[0].split('/')[-1].split('.')[0]
train_df = train.loc[train['Id'] == id_modified]
train_df

In [ ]:
id_modified1 = train.loc[train['Id'] == id_modified,'Pawpularity']
pawpularity= id_modified1.iloc[0]
pawpularity

In [ ]:
for i in range(5):
    id_modified=train_jpg[i].split('/')[-1].split('.')[0]
    id_modified_series = train.loc[train['Id'] == id_modified,'Pawpularity']
    pawpularity= id_modified_series.iloc[0]
    image= plt.imread(train_jpg[i]) 
    plt.imshow(image)
    title =train_jpg[i] + '   -   ' + str(pawpularity)
    plt.title(title) 
    plt.show()


In [ ]:
def image_to_show_based_on_pawpularity(desired_pawpularity):
    showing_image_numbers=5
    sample=train[train["Pawpularity"] == desired_pawpularity].sample(5).reset_index(drop=True)
    for i in range(5):
        id_modified = sample.iloc[i]['Id']
        root = '../input/petfinder-pawpularity-score/train/'
        extension = '.jpg'
        image_path = root + str(id_modified) + extension
        pawpularity=sample.iloc[i]['Pawpularity']
        image= plt.imread(image_path)
        plt.subplot(1, 5, i+1)
        title = pawpularity
        plt.title(title) 
        plt.imshow(image)
    plt.show()
   

In [ ]:
image_to_show_based_on_pawpularity(20)

In [ ]:
import os
import numpy as np
import pandas as pd
import cv2
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.regularizers import l2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.client import device_lib
import seaborn as sns
import matplotlib.pyplot as plt

Modeling_CNN

In [ ]:
train = pd.read_csv('../input/petfinder-pawpularity-score/train.csv')
test = pd.read_csv('../input/petfinder-pawpularity-score/test.csv')

In [ ]:
def train_id_to_path(x):
    return '/kaggle/input/petfinder-pawpularity-score/train/' + x + ".jpg"
def test_id_to_path(x):
    return '/kaggle/input/petfinder-pawpularity-score/test/' + x + ".jpg"
train = pd.read_csv('/kaggle/input/petfinder-pawpularity-score/train.csv')
train = train.drop(['Subject Focus', 'Eyes', 'Face', 'Near', 'Action', 'Accessory', 'Group', 'Collage', 'Human', 'Occlusion', 'Info', 'Blur'],axis=1)
test = pd.read_csv('/kaggle/input/petfinder-pawpularity-score/test.csv')
test = test.drop(['Subject Focus', 'Eyes', 'Face', 'Near', 'Action', 'Accessory', 'Group', 'Collage', 'Human', 'Occlusion', 'Info', 'Blur'],axis=1)
train["img_path"] = train["Id"].apply(train_id_to_path)
test["img_path"] = test["Id"].apply(test_id_to_path)

In [ ]:
train['two_bin_pawp'] = pd.qcut(train['Pawpularity'], q=2, labels=False)
train = train.astype({"two_bin_pawp": str})
train['four_bin_pawp'] = pd.qcut(train['Pawpularity'], q=4, labels=False)
train = train.astype({"four_bin_pawp": str})
train['ten_bin_pawp'] = pd.qcut(train['Pawpularity'], q=10, labels=False)
train = train.astype({"ten_bin_pawp": str})

In [ ]:
train4bin_stats = train.groupby('four_bin_pawp')
train4bin_stats.describe()

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
image_height = 128
image_width = 128
def path_to_eagertensor(image_path):
    raw = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(raw, channels=3)
    image = tf.cast(image, tf.float32) / 255.0
    image = tf.image.resize(image, (image_height, image_width))
    return image

In [ ]:
print(train['img_path'][0])

In [ ]:
og_example_image = plt.imread('../input/petfinder-pawpularity-score/train/0007de18844b0dbbb5e1f607da0606e0.jpg') 
print(og_example_image.shape)
plt.imshow(og_example_image)
plt.title('First Training Image') 
plt.show()

In [ ]:
example_image = path_to_eagertensor('../input/petfinder-pawpularity-score/train/0007de18844b0dbbb5e1f607da0606e0.jpg')


In [ ]:
print('type: ', type(example_image),'\n shape: ',example_image.shape)
plt.imshow(example_image)
plt.title('First Training Image - with preprocessing done by path_to_eagertensor()') 
plt.show()

In [ ]:
X = []
for img in train['img_path']:
    new_img_tensor = path_to_eagertensor(img)
    X.append(new_img_tensor)
    
print(type(X),len(X))
X = np.array(X)
print(type(X),X.shape)

In [ ]:
X_submission = []
for img in test['img_path']:
    new_img_tensor = path_to_eagertensor(img)
    X_submission.append(new_img_tensor)
    
print(type(X_submission),len(X_submission))
X_submission = np.array(X_submission)
print(type(X_submission),X_submission.shape)

In [ ]:
y = train['Pawpularity']
print(type(y))

In [ ]:
train4bin_stats = train.groupby('four_bin_pawp')
train4bin_stats.describe()

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

In [ ]:
import os
import numpy as np
import pandas as pd
import cv2
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.regularizers import l2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.client import device_lib
import seaborn as sns
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D, Conv1D, MaxPooling1D
from keras.layers import Dense, Activation, Dropout, Flatten
from sklearn.datasets import make_blobs
from sklearn.preprocessing import MinMaxScaler 
from keras.layers.advanced_activations import LeakyReLU
from keras.layers import BatchNormalization




In [ ]:
inputs = tf.keras.Input(shape=(image_height,image_width,3))
x = inputs

x = tf.keras.layers.Conv2D(filters = 16, kernel_size = (7,7), strides = (2,2),padding='valid',activation = 'relu')(x)
x = tf.keras.layers.Conv2D(filters = 32, kernel_size = (3,3), strides = (2,2), padding='same',activation = 'relu')(x)
x = tf.keras.layers.BatchNormalization()(x)


x = tf.keras.layers.Conv2D(filters = 64, kernel_size = (3,3), padding='same',activation = 'relu')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Conv2D(filters = 64, kernel_size = (3,3), strides = (2,2), padding='same',activation = 'relu')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(0.25)(x)

x = tf.keras.layers.Conv2D(filters = 128, kernel_size = (3,3), padding='same',activation = 'relu')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(x)
x = tf.keras.layers.Conv2D(filters = 128, kernel_size = (3,3),padding='same',activation = 'relu')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(0.25)(x)

x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(512, activation = "relu")(x)
x = tf.keras.layers.Dropout(0.5)(x)

output = tf.keras.layers.Dense(1)(x)

model = tf.keras.Model(inputs = inputs, outputs = output)


In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='Adam', loss='mse', metrics = [tf.keras.metrics.RootMeanSquaredError(name="rmse"), "mae", "mape"]) 

In [ ]:
data_augmentation = ImageDataGenerator(
    rotation_range = 15, 
    zoom_range = 0.15,
    width_shift_range = 0.2, 
    height_shift_range = 0.2, 
    shear_range = 0.1,
    horizontal_flip = True, 
    fill_mode = "nearest")

In [ ]:
history = model.fit(
    data_augmentation.flow(x_train,y_train,batch_size=32),
    validation_data = (x_test,y_test),
    epochs = 3
)

In [ ]:
def convert2onehot_4(labels):
    result = []
    for label in labels:
#         print(label)
        if label == '0':
#             print("label 1")
            result.append([1, 0, 0, 0])
        elif label == '1':
            result.append([0,1,0,0])
        elif label == '2':
            result.append([0,0,1,0])
        else:
#             print("label 0")
            result.append([0,0,0,1])
    return np.array(result)
x_train_array = np.array(x_train)
x_test_array = np.array(x_test)
y_train_onehot = convert2onehot_4(y_train)
y_test_onehot = convert2onehot_4(y_test)

In [ ]:
inputs1 = tf.keras.Input(shape=(image_height,image_width,3))
x1 = inputs1

x1 = tf.keras.layers.Conv2D(filters = 16, kernel_size = (7,7), strides = (2,2),padding='valid',activation = 'relu')(x1)
x1 = tf.keras.layers.Conv2D(filters = 32, kernel_size = (3,3), strides = (2,2), padding='same',activation = 'relu')(x1)
x1 = tf.keras.layers.BatchNormalization()(x1)


x1 = tf.keras.layers.Conv2D(filters = 64, kernel_size = (3,3), padding='same',activation = 'relu')(x1)
x1 = tf.keras.layers.BatchNormalization()(x1)
x1 = tf.keras.layers.Conv2D(filters = 64, kernel_size = (3,3), strides = (2,2), padding='same',activation = 'relu')(x1)
x1 = tf.keras.layers.BatchNormalization()(x1)
x1 = tf.keras.layers.Dropout(0.25)(x1)

x1 = tf.keras.layers.Conv2D(filters = 128, kernel_size = (3,3), padding='same',activation = 'relu')(x1)
x1 = tf.keras.layers.BatchNormalization()(x1)
x1 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(x1)
x1 = tf.keras.layers.Conv2D(filters = 128, kernel_size = (3,3),padding='same',activation = 'relu')(x1)
x1 = tf.keras.layers.BatchNormalization()(x1)
x1 = tf.keras.layers.Dropout(0.25)(x1)

x1 = tf.keras.layers.Flatten()(x1)
x1 = tf.keras.layers.Dense(512, activation = "relu")(x1)
x1 = tf.keras.layers.Dropout(0.5)(x1)

output1 = tf.keras.layers.Dense(4)(x1)

model1 = tf.keras.Model(inputs = inputs1, outputs = output1)



In [ ]:
model1.compile(optimizer='Adam', loss='mse', metrics = [tf.keras.metrics.RootMeanSquaredError(name="rmse"), "mae", "mape"]) 

In [ ]:
print(x_train_array.shape, y_train_onehot.shape,x_test_array.shape, y_test_onehot.shape)

In [ ]:
model1.summary()

In [ ]:
model1.fit(x_train_array, y_train_onehot, batch_size=16, epochs=3, validation_data=(x_test_array, y_test_onehot))

In [ ]:
cnn_pred = model.predict(X_submission)
print(X_submission.shape, type(X_submission))
print(cnn_pred.shape, type(cnn_pred))


In [ ]:
cnn = pd.DataFrame()
cnn['Id'] = test['Id']
cnn['Pawpularity'] = cnn_pred
cnn.to_csv('submission.csv',index=False)

In [ ]:
cnn.head(10)

In [ ]:
cnn_pred1 = model1.predict(X_submission)
print(X_submission.shape, type(X_submission))
print(cnn_pred1.shape, type(cnn_pred1))

In [ ]:
cnn1=pd.DataFrame(cnn_pred1)
cnn1